# Модульна контрольна робота 2: Класифікація, Частина 2

В даній роботі будемо продовжувати досліджувати та створювати класифікатор використовуючи метод k-nearest neighbors.

Необхідно буде створити класифікатор, який визначає, до якого жанру відноситься та чи інша пісня, використовуючи лише кількість слів, які зустрічаються в тексті пісні. В проекті буде необхідно:

1. Очистити і впорядкувати набір даних, який використовується для тестування моделі
2. Побудувати класифікатор k-найближчих сусідів
3. Перевірити класифікатор на даних

Для самоконтролю використовуються тести автогрейдера (from gofer.ok import check), як і в поперідній работі. Якщо Ви їх ще не використовували, то для їх викоритання необхідно інсталювати два додаткових модуля (через CMD.exe Prompt в ANACONDA NAVIGATOR) а саме:

- **okgrade** (pip install okgrade) https://okgrade.readthedocs.io/_/downloads/en/latest/pdf/
- **gofer** (pip install git+https://github.com/grading/gradememaybe.git) https://okgrade.readthedocs.io/en/latest/install/

Якщо Ви не бажаєти їх викоритовувати, то закоментуйте відповідний рядок і не використовуйте рядки тесту (наприклад check('tests/q2_1.py'), ...). В такому разі Ви можете переглянути наявні тести у відповідній директорії

In [1]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
import math
from datascience import *

# These lines set up the plotting functionality and formatting.
import matplotlib

matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots

plots.style.use('fivethirtyeight')
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter('ignore', UserWarning)

# These lines load the tests.
from gofer.ok import check

## Попередній огляд

У практичній роботі 5.5 ми виконали такі завдання:
1. У розділі 1 ми досліджували набір даних і розділили його на навчальні та тестові вибірки.
2. У розділі 2 ми розглянули приклад алгоритму класифікації k-Nearest Neightbors (k-NN).

**Якщо Ви недоробили практичну роботу 5.5 - поверніться та перегляньте її зараз. Це допоможе Вам у цій роботі. **

У цій роботі ми плануємо виконати такі завдання:
1. Визначите деякі ознаки.
2. Визначите функцію класифікатора, використовуючи обрані ознаки та навчальну вибірку.
3. Оцінемо ефективність класифікатора (відсоток правильних класифікацій) на тестовій вибірці.

Запустіть комірку нижче, щоб налаштувати проект.

In [2]:
lyrics = Table.read_table('lyrics.csv')

training_proportion = 11 / 16

num_songs = lyrics.num_rows
num_train = int(num_songs * training_proportion)
num_valid = num_songs - num_train

train_lyrics = lyrics.take(np.arange(num_train))
test_lyrics = lyrics.take(np.arange(num_train, num_songs))


def most_common(label, table):
    return table.group(label).sort('count', descending=True).column(label).item(0)

## 1. Ознаки

Тепер ми збираємося розширити наш класифікатор із практичної роботи 5.5, щоб використовувати більше двох ознак одночасно.

Евклідова відстань все ще має сенс з більш ніж двома ознаками. Для `n` різних ознак ми обчислюємо різницю між відповідними значеннями ознак для двох пісень, потім підводимо кожну з `n` різниць у квадрат, підсумовуємо отримані числа та беремо квадратний корінь із суми.

** <b>Завдання 1.1</b> ** <br/>

Напишіть функцію для обчислення евклідової відстані між двома **масивами (arrays)** ознак *довільної* (але однакової) довжини. Використуємо її, щоб обчислити відстань між першою піснею в навчальній вибірці та першою піснею в тестовому наборі, *використовуючи всі ознаки*. (Пам’ятайте, що `Title`, `Artist` та `Genre` пісень не являються ознаками.)

**Примітка.** Щоб перетворити рядкові об’єкти (row в таблиці створеної в datascience) на масиви (arrays), використовуйте `np.array`. Наприклад, якщо "t" була таблицею, "np.array(t.row(0))" перетворює 0 рядок "t" на масив.

In [3]:
def distance(features1, features2):
    """The Euclidean distance between two arrays of feature values."""
    return np.sqrt(np.sum(np.square(np.array(features1) - np.array(features2))))


distance_first_to_first = distance(test_lyrics.drop(0, 1, 2).row(0), train_lyrics.drop(0, 1, 2).row(0))
distance_first_to_first

0.1482277008140451

In [4]:
check("tests/q1_1.py")

### 1.1. Створення власного набору ознак

Однак, використання всіх ознак має деякі недоліки, ми їх обговорювали на лекції. Одним з явних недоліків є *обчислювальна вартість* — обчислення евклідових відстаней займає багато часу, якщо у нас є багато ознак. Ви могли помітити це в попередньому завданні! Крім того, не всі ознаки є інформативними, отже їх кількість варто зменшити.

Тож ми виберемо лише 20. Хотілося б обрати ознаки, які є дуже *дискримінативними*. Тобто ознаки, які дозволяють нам правильно класифікувати якомога більшу частину тестової вибірки. Цей процес вибору ознак, які забезпечать ефективну роботу класифікатора, іноді називають *вибором ознак* або ширше *конструюванням ознак* (https://en.wikipedia.org/wiki/Feature_engineering).

** <b>Завдання 1.1.1</b> ** <br/>


Перегляньте список ознак (атрибути таблиці `lyrics` окрім перших трьох). Виберіть 20 загальних слів, які, на вашу думку, допоможуть розрізнити пісні country та hip-hop. Обов’язково вибирайте слова, які вживаються достатньо часто, щоб кожна пісня містила принаймні одне з них. Але не обирайте лише 20 найчастіших... ви можете зробити набагато краще.

Можливо, Ви захочете повернутися до цього питання пізніше, щоб покращити свій набір ознак, коли Ви побачите, як оцінити свій класифікатор. Коли Ви вперше відповідаєте на це завданняя, приділіть деякий час перегляду ознак, але не більше 15 хвилин.

Наприклад, для свого набору ознак було обрано наступні ['love','like','ego','brick','creep','block','gun','gave','scream','in','will','we','with','what','out','from','let','back','was','got']. Це наведено для прикладу, але Ви створіть власний набір.

In [5]:
# Set my_20_features to an array of 20 features (strings that are column labels)
n_distinctive_features = 20

features_frequences = Table().with_columns(
    'Feature', lyrics.labels[3:],
    'Frequence', np.asarray(lyrics.columns[3:]).mean(axis=1)
)

my_20_features = features_frequences \
    .sort('Frequence', descending=True) \
    .column('Feature') \
    .take(range(n_distinctive_features))

print('Selected words:', my_20_features)

train_20 = train_lyrics.select(my_20_features)
test_20 = test_lyrics.select(my_20_features)

Selected words: ['i' 'the' 'you' 'and' 'a' 'to' 'it' 'not' 'me' 'my' 'in' 'is' 'that' 'of'
 'on' 'do' 'your' 'am' 'all' 'we']


Тест нижче перевіряє для гарантії, що Ви вибрали слова так, щоб принаймні одне з’явилося в кожній пісні. Якщо ви не можете знайти слова, які задовольняють цей тест лише завдяки інтуїції, спробуйте написати код, щоб роздрукувати назви пісень, які не містять жодного слова з вашого списку, а потім подивіться на слова, які вони містять.

In [6]:
check("tests/q1_1_1.py")

Далі давайте спробуємо класифікувати першу пісню з нашого тестового набору за допомогою обраних ознак. Ви можете переглянути пісню, запустивши комірку нижче. Як Ви думаєте, чи правильно буде її класифіковано?

In [7]:
print("Song:")
test_lyrics.take(0).select('Title', 'Artist', 'Genre').show()
print("Features:")
test_20.take(0).show()

Song:


Title,Artist,Genre
That Kind of Love,Alison Krauss,Country


Features:


i,the,you,and,a,to,it,not,me,my,in,is,that,of,on,do,your,am,all,we
0.010582,0.037037,0.005291,0.015873,0.015873,0.010582,0.037037,0.026455,0.005291,0.015873,0.005291,0.031746,0.026455,0.026455,0.010582,0.005291,0,0,0,0.042328


Як і раніше, ми хочемо знайти пісні в навчальній вибірці, які найбільше схожі на нашу тестову пісню. Ми обчислимо евклідові відстані від тестової пісні (використовуючи 20 обраних ознак) до всіх пісень в навчальній вибірці. Ви можете зробити це за допомогою циклу `for`, але щоб пришвидшити обчислення, Вам надається функція `fast_distances`, яка зробить це за Вас. Прочитайте її документацію, щоб переконатися, що Ви розумієте, що він робить. (Вам не потрібно читати код у тілі функції, якщо Ви цього не хочете.)

In [8]:
# Just run this cell to define fast_distances.

def fast_distances(test_row, train_rows):
    """An array of the distances between test_row and each row in train_rows.

    Takes 2 arguments:
      test_row: A row of a table containing features of one
        test song (e.g., test_20.row(0)).
      train_rows: A table of features (for example, the whole
        table train_20)."""
    assert train_rows.num_columns < 50, "Make sure you're not using all the features of the lyrics table."

    counts_matrix = np.asarray(train_rows.rows)
    distances = np.sqrt(np.sum(np.square(np.asarray(test_row) - counts_matrix), axis=1))

    return distances

** <b>Завдання 1.1.2</b> ** <br/>

Скористайтеся наданою вище функцією `fast_distances`, щоб обчислити відстань від першої пісні в тестовому наборі до всіх пісень в навчальній вибірці, **використовуючи ваш набір із 20 ознак**. Створіть нову таблицю під назвою `genre_and_distances` з одним рядком для кожної пісні в навчальній вибірці та двома стовпцями:
* `"Genre"` навчальної пісні (взяти з train_lyrics колонку `"Genre"`)
* `"Distance"` від першої пісні в тестовій вибірці (використати функцію `fast_distances`)

Переконайтеся, що `genre_and_distances` **відсортовано в порядку зростання за відстанню до першої тестової пісні**. (підказка - `Table().with_columns("Genre", ..., "Distance", ...).sort("Distance")`)

In [9]:
# The staff solution took about 4 lines of code, but it's not obviously.
genre_and_distances = Table().with_columns(
    'Genre', train_lyrics.column('Genre'),
    'Distance', fast_distances(test_20.row(0), train_20)
).sort('Distance')
genre_and_distances

Genre,Distance
Country,0.0489616
Hip-hop,0.0490157
Hip-hop,0.0517087
Hip-hop,0.0523167
Country,0.0526236
Hip-hop,0.0533627
Hip-hop,0.0551075
Hip-hop,0.0558699
Country,0.056612
Hip-hop,0.0566244


In [10]:
check("tests/q1_1_2.py")

** <b>Завдання 1.1.3</b> ** <br/>

Тепер проведіть класифікацію за 5 найближчими сусідами першої пісні в тестовому наборі. Тобто визначте її жанр, знайшовши найпоширеніший жанр серед 5 його найближчих сусідів відповідно до обчислених відстаней. Потім перевірте, чи Ваш класифікатор вибрав правильний жанр. (Залежно від обраних ознак, класифікатор може і неправильно класифікувати цю пісню, і це нормально.)

**Підказка** - для визначання жанру скористайтеся отриманою таблицею `genre_and_distances`. Візьміть 5 перших рядків (адже таблиця відсортована - `.take(range(5))`. Згрупуйте за жанром - `.group("Genre")`. Відсортуйте за зростянням - `.sort("count", descending = True)`. Витягніть колонку `"Genre"` - `.column("Genre")` та отримайте перший запис з колонки з назвою жанра - `.item(0)`.

Для визначення змінної my_assigned_genre_was_correct необхідно присвоїти їй результат порівняння отриманого спрогнозованого жанру - `my_assigned_genre` та жанру який реально визнчений у таблиці `test_lyrics` для цієї пісні - `test_lyrics.take(0).column("Genre").item(0)`.

In [11]:
# Set my_assigned_genre to the most common genre among these.
n_knn = 5
my_assigned_genre = genre_and_distances\
    .take(range(n_knn))\
    .group('Genre')\
    .sort('count', descending=True)\
    .column('Genre')\
    .item(0)

# Set my_assigned_genre_was_correct to True if my_assigned_genre
# matches the actual genre of the first song in the test set.
my_assigned_genre_was_correct = my_assigned_genre == test_lyrics.column('Genre').item(0)

print(f"The assigned genre, {my_assigned_genre}, was{' ' if my_assigned_genre_was_correct else ' not '}correct.")

The assigned genre, Hip-hop, was not correct.


In [12]:
check("tests/q1_1_3.py")

### 1.2. Функція класифікатора

Тепер ми можемо написати одну функцію, яка інкапсулює весь процес класифікації.

** <b>Завдання 1.2.1</b> ** <br/>

Напишіть функцію під назвою `classify`. Вона має прийняти такі чотири аргументи:
* Рядок ознак для класифікації пісні (наприклад, `test_20.row(0)`) - фактично *невідому* пісню.
* Таблиця тренувальної вибірки зі стовпцями значень для кожної з обраних ознак (наприклад, `train_20`).
* Масив класів (міститься в колонці `"Genre"`), який містить стільки елементів, скільки рядків у попередній таблиці, і в тому самому порядку.
* `k`, кількість сусідів для використання в класифікації.

Функція має повертати клас, який прогнозує класифікатор `k`- nearest neighbor для заданого рядку ознак (рядок `'Country'` або рядок `'Hip-hop'`).

**Підказка** - фактично запишіть вище пророблені кроки у функцію. Спочатку створіть таблицю як і раніше `genre_and_distances` а потім спрогнозуйте жанр як для змінної `my_assigned_genre` в завданні 1.1.3

In [13]:
def classify(test_row, train_rows, train_classes, k):
    """Return the most common class among k nearest neigbors to test_row."""
    distances = fast_distances(test_row, train_rows)
    genre_and_distances = Table().with_columns(
        'Genre', train_classes,
        'Distance', distances
    )
    most_common_class = genre_and_distances\
        .sort('Distance')\
        .take(range(k))\
        .group('Genre')\
        .sort('count', descending=True)\
        .column('Genre')\
        .item(0)
    return most_common_class

In [14]:
check("tests/q1_2_1.py")

** <b>Завдання 1.2.2</b> ** <br/>

Призначте змінній `grandpa_genre` жанр, передбачений вашим класифікатором для пісні "Grandpa Got Runned Over By A John Deere" у тестовому наборі, використовуючи **9 сусідів** і використовуючи ваші 20 ознак.

In [15]:
# The staff solution first defined a row object called grandpa_features.
grandpa_features = test_lyrics\
    .where('Title', 'Grandpa Got Runned Over By A John Deere')\
    .select(my_20_features)\
    .row(0)
grandpa_genre = classify(grandpa_features, train_20, train_lyrics['Genre'], k=9)
grandpa_genre

'Hip-hop'

In [16]:
check("tests/q1_2_2.py")

Нарешті, коли ми оцінюємо наш класифікатор, буде корисно мати функцію класифікації, яка спеціалізується на використанні фіксованого навчальної вибірки та фіксованого значення `k`.

** <b>Завдання 1.2.3</b> ** <br/>

Створіть функцію класифікації, яка приймає як аргумент row - рядок, що містить ваші 20 ознак, і класифікує цей рядок за допомогою алгоритму 5 найближчих сусідів із `train_20` в якості навчальної вибірки та `train_classes` з визнвченими класами. Тобто фактично напишіть функцію яка буде викликати класифікатор для будь якої пісні (row), а не як в попередньому завданні для чітко визнвченої пісні.

In [17]:
def classify_one_argument(row):
    return classify(row, train_20, train_lyrics['Genre'], k=5)


# When you're done, this should produce 'Hip-hop' or 'Country'.
classify_one_argument(test_20.row(0))

'Hip-hop'

In [18]:
check("tests/q1_2_3.py")

### 1.3. Оцінка класифікатора

Тепер, коли користуватися класифікатором стало легко, давайте перевіримо, наскільки він точний для всієї тестової вибірки.

** <b>Завдання 1.3.1</b> ** <br/> 

Використовуйте `classify_one_argument` і `apply`, щоб класифікувати кожну пісню в тестовому наборі. Назвіть ці припущення `test_guesses`. **Тоді** обчисліть частку правильних класифікацій. (`test_lyrics.select(...).apply(classify_one_argument)`) - додайте масив визначених Вами ознак. Для визначеня ж частки вірних класифікацій просто порявняйте спрогнозовані значення класів і реальні з тестової таблиці (`np.average(test_lyrics.column("Genre")`).

In [19]:
test_guesses = test_lyrics.select(my_20_features).apply(classify_one_argument)
proportion_correct = np.sum(test_lyrics['Genre'] == test_guesses) / num_valid
proportion_correct

0.64498141263940523

In [20]:
check("tests/q1_3_1.py")

На даний момент Ви пройшли один повний цикл розробки класифікатора. Давайте підсумуємо кроки:
1. З доступних даних оберіть тестові та навчальні вибірки.
2. Виберіть алгоритм, який ви збираєтеся використовувати для класифікації.
3. Визначте деякі ознаки.
4. Визначте функцію класифікатора, використовуючи ваші ознаки та навчальну вибірку.
5. Оцініть його результативність (частку правильних класифікацій) на тестовій вибірці.

Отже ми розглянули приклад для визначення жанру пісні - загальний приклад. Якщо Вам зрозумілі всі кроки варто застосувати отримані знання і навички для інших даних. Спробуйте адаптувати свій класифікатор для геологічних задач - наприклад для класифікації аномалій - magn_inv.csv, або для інших даних. Це буде максимально корисно для Вас. І хоча це необов'язково для отримання позитивної оцінки спробуйте отримати власний досвід!

## MAGN_INV

Read data from the `magn_ing.csv` file and split the dataset it into train and test sets

In [21]:
dataset = Table.read_table('magn_inv.csv')

training_proportion = 11 / 16

n_total = dataset.num_rows
n_train = int(n_total * training_proportion)
n_test = n_total - n_train

train_set = dataset.take(np.arange(n_train))
test_set = dataset.take(np.arange(n_train, n_total))

Use `classify` function from the previous task to apply classification on the dataset and find the best combination of features to separate anomalies from normal data

In [22]:
print(f"Available features: {dataset.labels[:-1]}")

test_res = test_set\
    .select(0, 1)\
    .apply(lambda row: classify(row, train_set.select(0, 1), train_set['Class'], k=2))
accuracy = np.sum(test_res == test_set['Class']) / n_test * 100
print(f"1. {accuracy:.2f}% - {dataset.select(0, 1).labels}")

test_res = test_set\
    .select(0, 2)\
    .apply(lambda row: classify(row, train_set.select(0, 2), train_set['Class'], k=2))
accuracy = np.sum(test_res == test_set['Class']) / n_test * 100
print(f"2. {accuracy:.2f}% - {dataset.select(0, 2).labels}")

test_res = test_set\
    .select(1, 2)\
    .apply(lambda row: classify(row, train_set.select(1, 2), train_set['Class'], k=2))
accuracy = np.sum(test_res == test_set['Class']) / n_test * 100
print(f"3. {accuracy:.2f}% - {dataset.select(1, 2).labels}")

Available features: ('Size', 'T', 'Grad')
1. 57.60% - ('Size', 'T')
2. 92.80% - ('Size', 'Grad')
3. 55.20% - ('T', 'Grad')


## Контроль

Ви закінчили з даною роботою! Аби перевірити, що всі завдання виконані вірно, виконайте наступні дії...
- **ВАЖЛИВО** Перш ніж щось робити, виберіть, **Save and Checkpoint** з меню `File`. Будь ласка, зробіть це спочатку, перш ніж запускати клітинку нижче,
- **запустіть усі тести та перевірте, чи всі вони пройшли успішно** (у наступній клітинці є перевірка для цього), 
- **Перегляньте notebook востаннє, перевірятися буде остаточний стан вашого notebook. Якщо ви вносили будь-які зміни, збережіть і запустіть всі рядки ще раз** Якщо ви вносили будь-які зміни, запустіть поллідовно всі рядки ще раз і збережіть **Save and Checkpoint** повторно.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import glob
from gofer.ok import grade_notebook

if not globals().get('__GOFER_GRADER__', False):
    display(grade_notebook('module_2.ipynb', sorted(glob.glob('tests/q*.py'))))